In [ ]:
git difftool --tool=b3 branch1..branch2



Rebase squash cherrypick automatically to create perfect feature branch?

Reset each file one at a time to make sure there aren't any unnecessary changes?

Use unit testing to warn about unnecessary modules?



In [ ]:
git log --graph --oneline --decorate --all

git log --graph --all

git show-branch --list



In [1]:
var D3Node = require('d3-node');
var d3n = new D3Node(); // initializes D3 with container element 
var d3 = d3n.d3;

var margin = {top: 25, right: 150, bottom: 25, left: 150},
    width = 960 - margin.left - margin.right,
    height = 500 - margin.top - margin.bottom;

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Documents/portal';
process.chdir(project)
var execSync = require('child_process').execSync;
var branches = execSync('git branch').toString().split(/\s*\n\s*\**\s*/ig);
var branch = execSync('git log --pretty=format:"%d %h %p" ' + branches.join(' ')).toString().split(/\s*\n\s*/ig);

nodeTree = [];
edges = [];
branch.forEach(b => {
    var commits = ((/\s*\(.*?\)\s*(.+)/ig).exec(b) || [b, b.trim()])[1].split(/\s+/ig);
    commits.forEach(r => {
        if(nodeTree.filter(d => d.name=='4a626f8').length == 0) {
            nodeTree[nodeTree.length] = {
                name: r
            }
        }
    });
    edges = edges.concat(commits.slice(1).map((c, i) => {
        return {source: c, target:commits[i]};
    }))
});

var nodeMap = {};
nodeTree.forEach(function(x) {
    if(typeof x.size == 'undefined') {
        x.size = 1;
    }
    nodeMap[x.name] = x;
});

var links = edges.map(function(x) {
    if(typeof nodeMap[x.source].children == 'undefined') {
        nodeMap[x.source].children = [];
    }
    nodeMap[x.source].children.push(nodeMap[x.target]);
    return { parent: nodeMap[x.target], source: nodeMap[x.source], target: nodeMap[x.target] };
});

console.log(nodeTree.filter(d => d.name=='4a626f8')[0]);

var tree = d3.hierarchy(nodeTree.filter(d => d.name=='4a626f8')[0], d => d.children);

var root = // partition(
    tree
        .sum(d => 1)
        .sort((a, b) => b.size - a.size || b.value - a.value)
//)
    .descendants().filter(d => d.depth < 2);

root.forEach(r => {
    r.name = r.data.name;
})

var tree = d3.tree().nodeSize([25, 150]);

var nodes = tree(d3.hierarchy(root[0], d => d.depth < 2 ? d.children : []))
    .descendants();

var links = nodes.slice(1);

var root = {
  name: 'root',
  children: nodes[0],
  hash: null,
  path: null,
  specificity: null,
  handler: null,
  data: {},
  isAux: false,
};
var fill = d3.scaleOrdinal(d3.schemeCategory20);
var svg = d3n.createSVG(width + margin.left + margin.right,
                       height + margin.top + margin.bottom)
var g = svg.append("g");

var x = d3.scaleLinear().range([0, height]);
var y = d3.scaleLinear().range([0, width]);

x.domain([d3.min(nodes, function(d) { return d.x; }), d3.max(nodes, function(d) { return d.x; })]);
y.domain([d3.min(nodes, function(d) { return d.y; }), d3.max(nodes, function(d) { return d.y; })]);

g.selectAll('.link')
  .data(links)
  .enter().append('path')
    .attr('class', 'link')
    .attr('d', d => `
        M${y(d.y)},${x(d.x)}
        C${y((d.y + d.parent.y) / 2)},
          ${x(d.x)} ${y((d.y + d.parent.y) / 2)},
          ${x(d.parent.x)} ${y(d.parent.y)},
          ${x(d.parent.x)}`);

// Declare the nodes
var node = g.selectAll('g.node')
  .data(nodes)
  .enter().append('g')
  .attr('class', 'node')
  .style("fill", function(d, i) { return fill(i); })
  .attr('transform', d => `translate(${y(d.y)},${x(d.x)})`);

node.append('circle')
  .attr('class', d => (d.data).isAux ? 'node-aux-route' : 'node-route')
  .attr('r', 6);

node.append('text')
  .attr('x', (d) => d.children ? -13 : 13)
  .attr('dy', '.35em')
  .attr('text-anchor', (d) => d.children ? 'end' : 'start')
  .text(d => (d.data).name)
  .attr('class', 'monospace');

// reset transform
g.attr('transform', 'translate(0, 0)');

g.attr("transform",
    "translate(" + (margin.left) + "," + (margin.top) + ")")

$$.svg(d3n.svgString());


{ name: '4a626f8',
  size: 1,
  children: [ { name: 'c9660c1', size: 1, children: [Object] } ] }
